In [123]:
import pandas as pd
import numpy as np

import itertools

module   = ['Science','Maths','Physics','Chemistry','English']
stgroups = range(1,11)
day      = ['Mon','Tue','Wed','Thu','Fri']
location = ['Hall A','Hall B','Hall C','Hall D','Hall E']
timing   = range(9,18)

ind = [('English', 1, 'Fri', 'Hall A', 14), ('English', 1, 'Tue', 'Hall D', 11), ('English', 1, 'Tue', 'Hall D', 11), ('English', 1, 'Tue', 'Hall D', 13), ('English', 1, 'Tue', 'Hall D', 14), ('Maths', 7, 'Fri', 'Hall A', 14), ('Chemistry', 3, 'Tue', 'Hall A', 13), ('Chemistry', 4, 'Thu', 'Hall A', 16), ('Chemistry', 8, 'Fri', 'Hall C', 13), ('Chemistry', 9, 'Wed', 'Hall D', 11), ('English', 2, 'Tue', 'Hall D', 12), ('English', 3, 'Thu', 'Hall E', 16), ('Maths', 3, 'Mon', 'Hall A', 15), ('Chemistry', 5, 'Tue', 'Hall E', 16)]
individual = ind
df = pd.DataFrame(individual, columns = ['Module' , 'StudentGroup', 'Weekday', 'Avenue', 'Time']) 
stg = df['StudentGroup'].unique()
fit = [] 
daily_classes = pd.DataFrame(list(itertools.product(module, day)),columns = ['Module','Weekday']).reset_index()
daily_hours = pd.DataFrame(list(itertools.product(timing, day)),columns = ['Time','Weekday']).reset_index()
 

In [124]:
    for i in stg:
        dfstg = df[df['StudentGroup'] == i].copy()
        #print(dfstg)
        
        #time conflicts within student group . minimize this
        dfstg['fit1'] = 0 
        #group by date and time and make there is only one entry for each.
        dff = dfstg[dfstg.groupby(['Weekday','Time'])['Time'].transform('count') > 1]
        #if entry is more than 1 then maximize this fitness for this individual
        dfstg.loc[dff.index,'fit1'] = 1
        
        
        # make sure classes cover all days. maximize this.
        dfstg['fit2'] = 1
        #get all lecture days for the group
        dff = dfstg.groupby(['Weekday'])['Weekday'].agg('count')
        y = dff.index.tolist()
        # assign fitness based on missing lecture days
        dfstg['fit2'] = len(y)/len(day)

        # make sure classes cover all times. maximize this.
        dfstg['fit3'] = 1
        #print(dfstg)
        #print('***************************************')
        #get all lecture time and days for the group
        dff = dfstg.groupby(['Weekday','Time'])['Weekday'].unique().agg('count')
        # assign fitness based on missing times per day
        dfstg['fit3'] = dff / len(daily_hours)
        #print(dfstg)
        #print('***************************************')        
        
        # make sure each lecture is not more than 2 in a singleday. maxmise this.
        dfstg['fit4'] = 1
        #print('***************************************')        
        #print(dfstg)       
        # take all classes which are more than 2 for each day
        dff = dfstg[dfstg.groupby(['Weekday','Module'])['Module'].transform('count') > 2]
        #print(dff)
        # for those classes alone have inverse fitness.
        if len(dff) > 0:
            dfstg.loc[dff.index,'fit4'] = 1 / len(dff)
        #print(dfstg)
        #print('***************************************')     
        
        # make sure each lecture is taken atleast one day. Maximize this
        dfstg['fit5'] = 1
        #print('***************************************')     
        #print(dfstg) 
        # take count of each module for each day
        dff = dfstg.groupby(['Weekday','Module'])['Module'].agg('count')
        y = dff.index.tolist()
        #print(dff)
        #print(y)
        dfstg['fit5'] = len(y) / (len(module) * len(day))
        #print(dfstg) 
        #print('***************************************')     
        
        # hall conflict at the same time and date with other student groups. minimize this
        dfstg['fit6'] = 0
        # select other student groups
        df1 = df[df['StudentGroup'] != i]
        # Count all records
        all_recs = len(df)
        # get conflicts using inner join
        conf_df = pd.merge(df1, dfstg, on=['Avenue','Weekday','Time'], how='inner').fillna(0)
        conflicts = len(conf_df)
        dfstg['fit6'] = conflicts / all_recs
        
        print(dfstg)
        fit1 = sum(dfstg.fit1.tolist())/len(dfstg)
        fit2 = sum(dfstg.fit2.tolist())/len(dfstg)
        fit3 = sum(dfstg.fit3.tolist())/len(dfstg)
        fit4 = sum(dfstg.fit4.tolist())/len(dfstg)
        fit5 = sum(dfstg.fit5.tolist())/len(dfstg)
        fit6 = sum(dfstg.fit6.tolist())/len(dfstg)
        
        fit.append((fit1, fit2, fit3, fit4, fit5, fit6))
    
    print(fit)
    fit = list(zip(*fit))
    print(fit)
    fit = [sum(i)/len(i) for i in fit]
    print(fit)

    Module  StudentGroup Weekday  Avenue  Time  fit1  fit2      fit3  fit4  \
0  English             1     Fri  Hall A    14     0   0.4  0.088889  1.00   
1  English             1     Tue  Hall D    11     1   0.4  0.088889  0.25   
2  English             1     Tue  Hall D    11     1   0.4  0.088889  0.25   
3  English             1     Tue  Hall D    13     0   0.4  0.088889  0.25   
4  English             1     Tue  Hall D    14     0   0.4  0.088889  0.25   

   fit5      fit6  
0  0.08  0.071429  
1  0.08  0.071429  
2  0.08  0.071429  
3  0.08  0.071429  
4  0.08  0.071429  
  Module  StudentGroup Weekday  Avenue  Time  fit1  fit2      fit3  fit4  \
5  Maths             7     Fri  Hall A    14     0   0.2  0.022222     1   

   fit5      fit6  
5  0.04  0.071429  
       Module  StudentGroup Weekday  Avenue  Time  fit1  fit2      fit3  fit4  \
6   Chemistry             3     Tue  Hall A    13     0   0.6  0.066667     1   
11    English             3     Thu  Hall E    16     0 